In [1]:
import os
import pandas as pd
import tensorflow as tf
import json
import pickle

In [2]:
data_path = '/media/nicolas/Lenovo/Projects/computer_vision_platzi/input'

In [3]:
train_data = os.path.join(data_path,'train')
test_data = os.path.join(data_path,'test')
json_train = os.path.join(data_path,'train.json')
json_test = os.path.join(data_path,'test.json')

In [4]:
train_data

'/media/nicolas/Lenovo/Projects/computer_vision_platzi/input/train'

In [5]:
data = open(json_test)
data = json.load(data)

In [7]:
data[0]

{'id': '948bc033-b0e6-45e0-aded-dd7009bd4c9568f05fb5-7b46-4036-99b0-f63252838ac1',
 'image': '39339a1089.jpg',
 'width': 640,
 'height': 478,
 'segmentation_url': '',
 'classification': [],
 'tags': [{'parent': None,
   'color': '#4ecc5e',
   'pos': {'x': 49, 'y': 6, 'w': 493, 'h': 453},
   'classes': [],
   'name': 'motorbike',
   'id': '6eedae51-0bbf-4862-b974-b1fb5e21fd93',
   'text': None,
   'type': 'bounding_box'}]}

In [29]:
def json_to_csv(json_file,type_file):
    data = open(json_file)
    data = json.load(data)
    csv_list = []
    for obj in data:
        width, height = obj['width'], obj['height']
        img_name = obj['image']
        for item in obj['tags']:
            class_name = item['name']
            xmin = item['pos']['x']
            ymin = item['pos']['y']
            ymax = item['pos']['y'] + item['pos']['h']
            xmax = item['pos']['x'] + item['pos']['w']
            value = (img_name,width,height,class_name,xmin,ymin,xmax,ymax)
        csv_list.append(value)
    df = pd.DataFrame(csv_list, columns = ['filename','width','height','class','xmin','ymin','xmax','ymax'])
    path = f'{data_path}/{type_file}/labels_{type_file}.csv'
    df.to_csv(path,index = False) 
    return path
    

In [30]:
csv_train = json_to_csv(json_train,'train')

In [31]:
csv_test = json_to_csv(json_test,'test')

In [34]:
"""
Usage:

# Create train data:
python generate_tfrecord.py --label=<LABEL> --csv_input=<PATH_TO_ANNOTATIONS_FOLDER>/train_labels.csv  --output_path=<PATH_TO_ANNOTATIONS_FOLDER>/train.record

# Create test data:
python generate_tfrecord.py --label=<LABEL> --csv_input=<PATH_TO_ANNOTATIONS_FOLDER>/test_labels.csv  --output_path=<PATH_TO_ANNOTATIONS_FOLDER>/test.record
"""

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf
import sys
sys.path.append("../../models/research")

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

# TO-DO replace this with label map
# for multiple labels add more else if statements
def class_text_to_int(row_label):
    if row_label == 'motorbike':  # 'ship':
        return 1
    elif row_label == 'car':
        return 2
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


output_path = 'test.record'
csv_input = f"{data_path}/labels_test.csv"
writer = tf.io.TFRecordWriter(output_path)
path = os.path.join(test_data)
examples = pd.read_csv(csv_input)
grouped = split(examples, 'filename')
for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())

writer.close()
output_path = os.path.join(output_path)
print('Successfully created the TFRecords: {}'.format(output_path))

Successfully created the TFRecords: test.record
